In [1]:
import gurobipy as gp
from gurobipy import GRB
import tomllib as tml
import numpy as np

Options looks like:
```
options = {
    "WLSACCESSID": "203dec48-e3f8-46ac-0184-92d7d6ded944",
    "WLSSECRET": "a080cce8-4e01-4e36-955e-61592c5630db",
    "LICENSEID": 12127,
}
```

In [2]:
# get gurobi credentials
options = tml.load(open("license.toml", "rb"))

## Test with MPS File
I made a MPS file by solving LP.mod (written by Quan Luu) with GLPK for Windows.

In [3]:
m = gp.read("model.mps")
m.reset()
m.optimize()

Restricted license - for non-production use only - expires 2025-11-24
Read MPS format model from file model.mps
Reading time = 0.00 seconds
LP: 757 rows, 729 columns, 2160 nonzeros
Discarded solution information
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 2600X Six-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 757 rows, 729 columns and 2160 nonzeros
Model fingerprint: 0xcc557f7f
Variable types: 0 continuous, 729 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-03, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Found heuristic solution: objective -0.0927000
Presolve removed 27 rows and 0 columns
Presolve time: 0.01s
Presolved: 730 rows, 729 columns, 2160 nonzeros
Variable types: 0 continuous, 729 integer (729 binary)
Found heuristic solution: obje

## Problem Setup
I'm going to try and convert this outright to a Gurobi model.

In [4]:
# establish env (must close)
env = gp.Env(params=options)

# establish model (must close)
model = gp.Model(env=env)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2527858
Academic license 2527858 - for non-commercial use only - registered to mb___@ur.rochester.edu


In [5]:
# parameters
n = 27
k = 6
V = {i for i in range(n)}

prob = np.asarray(
       [0.0137, 0.0298, 0.0137, 
        0.0298, 0.0651, 0.0298, 
        0.0137, 0.0298, 0.0137, 
        0.0298, 0.0651, 0.0298,
        0.0651, 0.1422, 0.0651,
        0.0298, 0.0651, 0.0298,
        0.0137, 0.0298, 0.0137,
        0.0298, 0.0651, 0.0298,
        0.0137, 0.0298, 0.0137]
    )

reward = np.asarray([
    [ 1.0,   0.5,   0.0,   0.5,   0.0,  -0.5,   0.0,  -0.5,  -1.0,   0.5,   0.0, - 0.5,   0.0,  -0.5,  -1.0,  -0.5,  -1.0,  -1.5,   0.0,  -0.5,  -1.0,  -0.5,  -1.0,  -1.5,  -1.0,  -1.5,  -2.0],
    [ 0.5,   1.0,   0.5,   0.0,   0.5,   0.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,   0.0,  -0.5,   0.0,  -0.5,  -1.0,  -0.5,  -1.0,  -0.5,   0.0,  -0.5,  -1.0,  -0.5,  -1.0,  -1.5,  -1.0,  -1.5],
    [ 0.0,   0.5,   1.0,  -0.5,   0.0,   0.5,  -1.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,  -1.0,  -0.5,   0.0,  -1.5,  -1.0,  -0.5,  -1.0,  -0.5,   0.0,  -1.5,  -1.0,  -0.5,  -2.0,  -1.5,  -1.0],
    [ 0.5,   0.0,  -0.5,   1.0,   0.5,   0.0,   0.5,   0.0,  -0.5,   0.0,  -0.5,  -1.0,   0.5,   0.0,  -0.5,   0.0,  -0.5,  -1.0,  -0.5,  -1.0,  -1.5,   0.0,  -0.5,  -1.0,  -0.5,  -1.0,  -1.5],
    [ 0.0,   0.5,   0.0,   0.5,   1.0,   0.5,   0.0,   0.5,   0.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,   0.0,  -0.5,   0.0,  -0.5,  -1.0,  -0.5,  -1.0,  -0.5,   0.0,  -0.5,  -1.0,  -0.5,  -1.0],
    [-0.5,   0.0,   0.5,   0.0,   0.5,   1.0,  -0.5,   0.0,   0.5,  -1.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,  -1.0,  -0.5,   0.0,  -1.5,  -1.0,  -0.5,  -1.0,  -0.5,   0.0,  -1.5,  -1.0,  -0.5],
    [ 0.0,  -0.5,  -1.0,   0.5,   0.0,  -0.5,   1.0,   0.5,   0.0,  -0.5,  -1.0,  -1.5,   0.0,  -0.5,  -1.0,  0.5,    0.0,  -0.5,  -1.0,  -1.5,  -2.0,  -0.5,  -1.0,  -1.5,   0.0,  -0.5,  -1.0],
    [-0.5,   0.0,  -0.5,   0.0,   0.5,   0.0,   0.5,   1.0,   0.5,  -1.0,  -0.5,  -1.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,   0.0,  -1.5,  -1.0,  -1.5,  -1.0,  -0.5,  -1.0,  -0.5,   0.0,  -0.5],
    [-1.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,   0.0,   0.5,   1.0,  -1.5,  -1.0,  -0.5,  -1.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,  -2.0,  -1.5,  -1.0,  -1.5,  -1.0,  -0.5,  -1.0,  -0.5,   0.0],
    [ 0.5,   0.0,  -0.5,   0.0,  -0.5,  -1.0,  -0.5,  -1.0,  -1.5,   1.0,   0.5,   0.0,   0.5,   0.0,  -0.5,   0.0,  -0.5,  -1.0,   0.5,   0.0,  -0.5,   0.0,  -0.5,  -1.0,  -0.5,  -1.0,  -1.5],
    [ 0.0,   0.5,   0.0,  -0.5,   0.0,  -0.5,  -1.0,  -0.5,  -1.0,   0.5,   1.0,   0.5,   0.0,   0.5,   0.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,   0.0,  -0.5,   0.0,  -0.5,  -1.0,  -0.5,  -1.0],
    [-0.5,   0.0,   0.5,  -1.0,  -0.5,   0.0,  -1.5,  -1.0,  -0.5,   0.0,   0.5,   1.0,  -0.5,   0.0,   0.5,  -1.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,  -1.0,  -0.5,   0.0,  -1.5,  -1.0,  -0.5],
    [ 0.0,  -0.5,  -1.0,   0.5,   0.0,  -0.5,   0.0,  -0.5,  -1.0,   0.5,   0.0,  -0.5,   1.0,   0.5,   0.0,   0.5,   0.0,  -0.5,   0.0,  -0.5,  -1.0,   0.5,   0.0,  -0.5,   0.0,  -0.5,  -1.0],
    [-0.5,   0.0,  -0.5,   0.0,   0.5,   0.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,   0.0,   0.5,   1.0,   0.5,   0.0,   0.5,   0.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,   0.0,  -0.5,   0.0,  -0.5],
    [-1.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,  -1.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,   0.0,   0.5,   1.0,  -0.5,   0.0,   0.5,  -1.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,  -1.0,  -0.5,   0.0],
    [-0.5,  -1.0,  -1.5,   0.0,  -0.5,  -1.0,   0.5,   0.0,  -0.5,   0.0,  -0.5,  -1.0,   0.5,   0.0,  -0.5,   1.0,   0.5,   0.0,  -0.5,  -1.0,  -1.5,   0.0,  -0.5,  -1.0,   0.5,   0.0,  -0.5],
    [-1.0,  -0.5,  -1.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,   0.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,   0.0,   0.5,   1.0,   0.5,  -1.0,  -0.5,  -1.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,   0.0],
    [-1.5,  -1.0,  -0.5,  -1.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,  -1.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,   0.0,   0.5,   1.0,  -1.5,  -1.0,  -0.5,  -1.0,  -0.5,   0.0,  -0.5,   0.0,   0.5],
    [ 0.0,  -0.5,  -1.0,  -0.5,  -1.0,  -1.5,  -1.0,  -1.5,  -2.0,   0.5,   0.0,  -0.5,   0.0,  -0.5,  -1.0,  -0.5,  -1.0,  -1.5,   1.0,   0.5,   0.0,   0.5,   0.0,  -0.5,   0.0,  -0.5,  -1.0],
    [-0.5,   0.0,  -0.5,  -1.0,  -0.5,  -1.0,  -1.5,  -1.0,  -1.5,   0.0,   0.5,   0.0,  -0.5,   0.0,  -0.5,  -1.0,  -0.5,  -1.0,   0.5,   1.0,   0.5,   0.0,   0.5,   0.0,  -0.5,   0.0,  -0.5],
    [-1.0,  -0.5,   0.0,  -1.5,  -1.0,  -0.5,  -2.0,  -1.5,  -1.0,  -0.5,   0.0,   0.5,  -1.0,  -0.5,   0.0,  -1.5,  -1.0,  -0.5,   0.0,   0.5,   1.0,  -0.5,   0.0,   0.5,  -1.0,  -0.5,   0.0],
    [-0.5,  -1.0,  -1.5,   0.0,  -0.5,  -1.0,  -0.5,  -1.0,  -1.5,   0.0,  -0.5,  -1.0,   0.5,   0.0,  -0.5,   0.0,  -0.5,  -1.0,   0.5,   0.0,  -0.5,   1.0,   0.5,   0.0,   0.5,   0.0,  -0.5],
    [-1.0,  -0.5,  -1.0,  -0.5,   0.0,  -0.5,  -1.0,  -0.5,  -1.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,   0.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,   0.0,   0.5,   1.0,   0.5,   0.0,   0.5,   0.0],
    [-1.5,  -1.0,  -0.5,  -1.0,  -0.5,   0.0,  -1.5,  -1.0,  -0.5,  -1.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,  -1.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,   0.0,   0.5,   1.0,  -0.5,   0.0,   0.5],
    [-1.0,  -1.5,  -2.0,  -0.5,  -1.0,  -1.5,   0.0,  -0.5,  -1.0,  -0.5,  -1.0,  -1.5,   0.0,  -0.5,  -1.0,   0.5,   0.0,  -0.5,   0.0,  -0.5,  -1.0,   0.5,   0.0,  -0.5,   1.0,   0.5,   0.0],
    [-1.5,  -1.0,  -1.5,  -1.0,  -0.5,  -1.0,  -0.5,   0.0,  -0.5,  -1.0,  -0.5,  -1.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,   0.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,   0.0,   0.5,   1.0,   0.5],
    [-2.0,  -1.5,  -1.0,  -1.5,  -1.0,  -0.5,  -1.0,  -0.5,   0.0,  -1.5,  -1.0,  -0.5,  -1.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,  -1.0,  -0.5,   0.0,  -0.5,   0.0,   0.5,   0.0,   0.5,   1.0],
])

In [6]:
# Create Hess Variables
# GPL: var x{V, V} >= 0, <= 1, binary;
x = model.addVars(V, V, ub=1, vtype=GRB.BINARY)

In [7]:
# state objective
# GPL: maximize EP: sum{i in V} PROB[i] * sum{j in V} x[i, j] * REWARD[i, j];
# gp.quicksum( prob[i] * x[i][j] * reward[i][j] for i in V for j in V )
objective = gp.quicksum( gp.quicksum( (prob[i] * x[i,j] * reward[i][j]) for j in V) for i in V )
model.setObjective(objective, GRB.MAXIMIZE)

In [8]:
# add constraints

# /* there are exactly k buckets */
# kBucketConstr: sum{j in V} x[j, j] = k;
k_bucket = gp.quicksum( (x[j,j]) for j in V ) == k
model.addConstr(k_bucket)

# /* a state can only belong to one bucket */
# uniqueBucketConstr{i in V}: sum{j in V} x[i, j] = 1;
unique_bucket = (gp.quicksum( (x[i,j]) for j in V ) == 1 for i in V)
model.addConstrs(unique_bucket)

# /* a state cannot belong to a non-existant bucket */
# nonexBucketConstr{i in V, j in V}: x[i, j] <= x[j, j];
nonex_bucket = ( (x[i,j] <= x[j,j]) for i in V for j in V )
model.addConstrs(nonex_bucket)

model.update()

In [9]:
# can we solve?
model.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 2600X Six-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Academic license 2527858 - for non-commercial use only - registered to mb___@ur.rochester.edu
Optimize a model with 757 rows, 729 columns and 2160 nonzeros
Model fingerprint: 0xbd42aa8b
Variable types: 0 continuous, 729 integer (729 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-03, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Found heuristic solution: objective -0.0927000
Presolve removed 27 rows and 0 columns
Presolve time: 0.01s
Presolved: 730 rows, 729 columns, 2160 nonzeros
Variable types: 0 continuous, 729 integer (729 binary)
Found heuristic solution: objective 0.3946500

Root relaxation: objective 6.790500e-01, 88 iterations, 0.00 seconds (0.00 work units)

    Nodes  